In [5]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the dataset
data = np.load('lab2_dataset.npz')
train_feats = torch.tensor(data['train_feats']) # 44730 examples, 11 frames/example, 40 mfcc bins
test_feats = torch.tensor(data['test_feats']) # 4773 examples, 11 frames/example, 40 mfcc bins
train_labels = torch.tensor(data['train_labels']) # 44730 training data labels 
test_labels = torch.tensor(data['test_labels']) # 4773 testing data labels 
phone_labels = data['phone_labels'] # 48 labels for the 48 classifications

# Set up the dataloaders
train_dataset = torch.utils.data.TensorDataset(train_feats, train_labels)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)

test_dataset = torch.utils.data.TensorDataset(test_feats, test_labels)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=False)

# Define the model architecture
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        # TODO: Fill in the model's layers here
        val = [440, 400, 300, 250, 200, 150, 100, 75, 48]
        self.lin1 = nn.Linear(val[0], val[1])
        #self.batchNorm1 = nn.BatchNorm1d(val[1])
        self.lin2 = nn.Linear(val[1], val[2])
        #self.batchNorm2 = nn.BatchNorm1d(val[2])
        self.lin3 = nn.Linear(val[2], val[3])
        #self.batchNorm3 = nn.BatchNorm1d(val[3])
        self.lin4 = nn.Linear(val[3], val[4])
        #self.batchNorm4 = nn.BatchNorm1d(val[4])
        self.lin5 = nn.Linear(val[4], val[5])
        #self.batchNorm5 = nn.BatchNorm1d(val[5])
        self.lin6 = nn.Linear(val[5], val[6])
        self.lin7 = nn.Linear(val[6], val[7])
        self.lin7 = nn.Linear(val[7], val[8])
        self.relu = nn.ReLU()


    def forward(self, x):
        # TODO: Fill in the forward pass here
        #print(x.shape)
        x = torch.reshape(x, (-1, 11*40))
        #print(x.shape)
        x = self.relu(self.lin1(x))
        #x = self.batchNorm1(x)
        x = self.relu(self.lin2(x))
        #x = self.batchNorm2(x)
        x = self.relu(self.lin3(x))
        #x = self.batchNorm3(x)
        x = self.relu(self.lin4(x))
        #x = self.batchNorm4(x)
        x = self.relu(self.lin5(x))
        #x = self.batchNorm5(x)
        x = self.relu(self.lin6(x))
        x = self.lin7(x)
        return x

# Instantiate the model, loss function, and optimizer
model = MyModel()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

def train_network(model, train_loader, criterion, optimizer):
    # TODO: fill in
    numEpochs = 8
    for epoch in range(numEpochs):
        print('Epoch', epoch, 'running...')
        for i, (inputs, labels) in enumerate(train_loader, 0):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

def test_network(model, test_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Test accuracy: %d %%' % (100 * correct / total))

train_network(model, train_loader, criterion, optimizer)
test_network(model, test_loader)


Epoch 0 running...
Epoch 1 running...
Epoch 2 running...
Epoch 3 running...
Epoch 4 running...
Epoch 5 running...
Epoch 6 running...
Epoch 7 running...
Test accuracy: 55 %
